# Analytical Assignment

This Jupyter Notebook is an analysis of a bank's login by financial aggregators. The requirements are:

Produce a presentation for Banc ABC’s CISO and her team covering the following questions as well as any other insights you may deem important to bring to Banc ABC’s attention.
1.	Which aggregators are accessing Banc ABC’s systems?
2.	How much volume are the aggregators sending?
3.	How many individual user accounts are being accessed by the aggregators?
4.	What is the login success rate of these aggregators and is this in line with what would be expected in your opinion?
5.	What is the average number of transactions each aggregator sends per 10min interval?
6.	What is the maximum number of transactions each aggregator sends per 10min interval?
7.	What would be the impact of the CISO’s proposal to limit each aggregator to 1 login per account per 10 min interval?


# Table of Content

## [Load Data](#load-data)
    
## [Data Cleaning](#data-cleaning)

## [Analysis:](#analysis)

    
   ###       [Aggregators accessing Banc ABC](#q1)
    
   ###       [Volume sent by Aggregators](#q2)
    
   ###       [User accounts accessed](#q3)
    
   ###       [Login success rate](#q4)
    
   ###       [Avg transactions per 10 min](#q5)
    
   ###       [Max transactions per 10 min](#q6)
    
   ###       [Impact of CISO proposal](#q7)

## Load Data <a class=anchor id=load-data></a>

In [197]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

df_aggregators = pd.read_csv('C:\MIT micromaster\F5 takehome\Aggregator IP list.csv')
df_logins = pd.read_csv('C:\MIT micromaster\F5 takehome\Login Transactions.csv')


In [198]:
df_aggregators.head(10)

,Unnamed: 0,ip,Aggregator
0,0,54.208.59.10,AWS
1,1,54.88.74.128,AWS
2,2,54.88.202.28,AWS
3,3,52.70.160.54,FinTech
4,8,52.44.118.176,FinTech
5,9,52.70.161.90,FinTech
6,10,52.0.123.48,FinTech
7,11,206.108.41.101,Insight
8,12,206.108.41.102,Insight
9,13,206.108.41.103,Insight


In [199]:
df_logins.head(10)

,Unnamed: 0,IP,LoginSuccess,AccountName,timestamp
0,0,54.208.59.10,Fail,756bb790d96873a,1.540700e+12
1,1,206.108.41.103,Success,7b3eb9367400c1b,1.540700e+12
2,2,216.34.61.115,Success,abfa625be56908a,1.540700e+12
3,3,52.44.118.176,Fail,dad678e942a965c,1.540700e+12
4,4,64.41.181.29,Success,e3014e35d5ec49b,1.540700e+12
5,5,54.88.202.28,Fail,55fb395a2f89ebd,1.540700e+12
6,6,64.41.181.105,Fail,d15dbb3d529e52a,1.540700e+12
7,7,54.88.202.28,Fail,c593e6f46733a61,1.540700e+12
8,8,206.108.41.103,Success,76bd4ea082f40ae,1.540700e+12
9,9,54.88.74.128,Fail,ddfb8256e16fa12,1.540700e+12


## Data Cleaning <a class=anchor id=data-cleaning></a>

In [200]:
df_aggregators.info()  ##there are 315 entries all non null for each column
df_aggregators.describe()
df_aggregators.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 315 entries, 0 to 314
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  315 non-null    int64 
 1   ip          315 non-null    object
 2   Aggregator  315 non-null    object
dtypes: int64(1), object(2)
memory usage: 7.5+ KB


Index(['Unnamed: 0', 'ip', 'Aggregator'], dtype='object')

In [201]:
df_logins.info()
df_logins.describe()
df_logins.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 5 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   Unnamed: 0    1048575 non-null  int64  
 1   IP            1048575 non-null  object 
 2   LoginSuccess  1048575 non-null  object 
 3   AccountName   1048575 non-null  object 
 4   timestamp     1048575 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 40.0+ MB


Index(['Unnamed: 0', 'IP', 'LoginSuccess', 'AccountName', 'timestamp'], dtype='object')

## Analysis <a class=anchor id=analysis></a>

### Aggregators accessing Banc ABC <a class=anchor id='q1'></a>

1. AWS
2. FinTech
3. Insight
4. YoungOnes
5. PayTM
6. Funtown 

**(funtown appeared in two forms in the column so using the string function of lower case I made a consistent string type data series in that column to get the distinct aggregators accessing Banc ABC system)**

In [202]:
df_aggregators.Aggregator.unique()

array(['AWS', 'FinTech', 'Insight', 'YoungOnes', 'PayTM', 'Funtown',
       'FunTown'], dtype=object)

The code below explained:
1. The first line of code performs string function of lower-casing the Aggregator column
2. The name of column 'ip' changed to 'IP' for better understanding through consistency in the two tables when merging them.

In [203]:
df_aggregators['Aggregator'] = df_aggregators.Aggregator.str.lower() 

df_aggregators.rename(columns = {'ip': 'IP'}, inplace = True) 

df_aggregators.columns

Index(['Unnamed: 0', 'IP', 'Aggregator'], dtype='object')

### Volume sent by aggregators <a class=anchor id=q2></a>

#### How much volume are the aggregators sending?

1. I start by merging the two tables over the common key IP. 
2. I performed a left join of both tables to ensure I do not miss the IP values which are not whitelisted.
3. I replaces the NaN values of the aggregator column in the merged table with 'unfamiliar IP'.
4. I perform a groupby on the aggregator table to see how many IPs are allowed by the banc to get a better understanding of        the unfamiliar IP and to see if the merge was successful. I also studied how many IP addresses were allowed from YoungOnes      because they had the highest number of white-listed IP addresses.
5. From the merged table I get a new table which is grouped by the aggregator and the type of login attempt(fail, success).
6. I further group this grouped table to get the total login attempts made by each aggregator.

In [204]:
df_merged = pd.merge(df_logins, df_aggregators, how = 'left', on = ['IP'])

df_merged.head(15)

,Unnamed: 0_x,IP,LoginSuccess,AccountName,timestamp,Unnamed: 0_y,Aggregator
0,0,54.208.59.10,Fail,756bb790d96873a,1.540700e+12,0.0,aws
1,1,206.108.41.103,Success,7b3eb9367400c1b,1.540700e+12,13.0,insight
2,2,216.34.61.115,Success,abfa625be56908a,1.540700e+12,35.0,youngones
3,3,52.44.118.176,Fail,dad678e942a965c,1.540700e+12,8.0,fintech
4,4,64.41.181.29,Success,e3014e35d5ec49b,1.540700e+12,261.0,youngones
5,5,54.88.202.28,Fail,55fb395a2f89ebd,1.540700e+12,2.0,aws
6,6,64.41.181.105,Fail,d15dbb3d529e52a,1.540700e+12,27.0,youngones
7,7,54.88.202.28,Fail,c593e6f46733a61,1.540700e+12,2.0,aws
8,8,206.108.41.103,Success,76bd4ea082f40ae,1.540700e+12,13.0,insight
9,9,54.88.74.128,Fail,ddfb8256e16fa12,1.540700e+12,1.0,aws


In [205]:
df_merged = pd.merge(df_logins, df_aggregators, how = 'left', on = ['IP'])

df_merged.head(15)

df_merged[df_merged['Aggregator'].isnull()]

df_merged['Aggregator'] = df_merged.Aggregator.replace(np.nan,'unfamiliar IP',regex = True)

df_merged.head(10)


,Unnamed: 0_x,IP,LoginSuccess,AccountName,timestamp,Unnamed: 0_y,Aggregator
0,0,54.208.59.10,Fail,756bb790d96873a,1.540700e+12,0.0,aws
1,1,206.108.41.103,Success,7b3eb9367400c1b,1.540700e+12,13.0,insight
2,2,216.34.61.115,Success,abfa625be56908a,1.540700e+12,35.0,youngones
3,3,52.44.118.176,Fail,dad678e942a965c,1.540700e+12,8.0,fintech
4,4,64.41.181.29,Success,e3014e35d5ec49b,1.540700e+12,261.0,youngones
5,5,54.88.202.28,Fail,55fb395a2f89ebd,1.540700e+12,2.0,aws
6,6,64.41.181.105,Fail,d15dbb3d529e52a,1.540700e+12,27.0,youngones
7,7,54.88.202.28,Fail,c593e6f46733a61,1.540700e+12,2.0,aws
8,8,206.108.41.103,Success,76bd4ea082f40ae,1.540700e+12,13.0,insight
9,9,54.88.74.128,Fail,ddfb8256e16fa12,1.540700e+12,1.0,aws


In [206]:
df_groupedagg = df_aggregators.groupby(['Aggregator', 'IP'])['IP'].count().reset_index(name = 'count')

IP_allowed = df_groupedagg.groupby('Aggregator')['count'].sum().reset_index(name = 'num of IP allowed')

IP_allowed


,Aggregator,num of IP allowed
0,aws,4
1,fintech,10
2,funtown,3
3,insight,15
4,paytm,34
5,youngones,249


In [207]:
agg_vol = df_merged.groupby(['Aggregator', 'LoginSuccess'])['LoginSuccess'].count().reset_index(name = 'count')

agg_vol

,Aggregator,LoginSuccess,count
0,aws,Fail,516218
1,fintech,Fail,54366
2,fintech,Success,316
3,funtown,Fail,5
4,funtown,Success,3
5,insight,Fail,6813
6,insight,Success,133942
7,paytm,Fail,18443
8,paytm,Success,3072
9,unfamiliar IP,Fail,25646


In [208]:
agg_total_vol = agg_vol.groupby('Aggregator')['count'].sum().reset_index(name = 'volume being sent')
agg_total_vol_sorted = agg_total_vol.sort_values(by = 'volume being sent', ascending = False)
agg_total_vol_sorted

,Aggregator,volume being sent
0,aws,516218
6,youngones,219039
3,insight,140755
5,unfamiliar IP,96358
1,fintech,54682
4,paytm,21515
2,funtown,8


### User accounts accessed <a class=anchor id=q3></a>

1. First I analyzed the total number of unique accounts to get an idea about how many accounts are unique in the data
2. I grouped the data by aggregator, account name (to keep its uniqueness for each aggregator) and loginsuccess for each          account.
3. then I added a condition to the count of the new table to count the failed attempts as zero as failed attempt means account    was not accessed. I kept the failed attempts so as to make the table understandable especially for AWS which has no            successful logins.
4. With failed attempts counted as zero only successful attempts were truly counted.
5. I then grouped this data by the sum of the count of successful individual accounts access for each aggregator.

In [209]:
total_num_unique_useraccounts = df_merged.AccountName.nunique()

total_num_unique_useraccounts

908039

In [210]:
user_acc_accessed = df_merged.groupby(['Aggregator', 'AccountName', 'LoginSuccess'])['LoginSuccess'].count().reset_index(name = 'count')

user_acc_accessed.loc[user_acc_accessed['LoginSuccess'] == 'Fail', ['count']] = 0

user_acc_accessed.loc[user_acc_accessed['Aggregator'] == 'funtown']

,Aggregator,AccountName,LoginSuccess,count
549876,funtown,1884b16e341e893,Fail,0
549877,funtown,9784de6a30fc497,Fail,0
549878,funtown,af6630b19a35bd3,Success,1
549879,funtown,b9fa0dfb107b6fb,Fail,0
549880,funtown,b9fa0dfb107b6fb,Success,1
549881,funtown,e321081509e0bac,Success,1


In [211]:
total_user_acc_accessed = user_acc_accessed.groupby('Aggregator')['count'].sum().reset_index(name = 'total accounts accessed')

total_user_acc_accessed_sorted = total_user_acc_accessed.sort_values(by = 'total accounts accessed', ascending = False)

total_user_acc_accessed_sorted

,Aggregator,total accounts accessed
6,youngones,155549
3,insight,133942
5,unfamiliar IP,70712
4,paytm,3072
1,fintech,316
2,funtown,3
0,aws,0


### Login success rate <a class=anchor id=q4></a>

## What is the login success rate of these aggregators and is this in line with what would be expected in your opinion?

1. I first grouped the merged table by Aggregator and LoginSuccess and find the percentage of each type of LoginSuccess: 'Fail'    or 'Success' from the total login attempts by each aggregator.
2. For simplicity I redefined the failed login attempts percentages as 0.
3. The final table shows (in descending order) the login success rate as a percentage of the total login attempts by each          aggregator.

In [212]:
login_success = df_merged.groupby(['Aggregator', 'LoginSuccess'])['LoginSuccess'].count().groupby(level = 0).apply(lambda x: x*100/x.sum()).reset_index(name = 'login success rate')

login_success


,Aggregator,LoginSuccess,login success rate
0,aws,Fail,100.000000
1,fintech,Fail,99.422113
2,fintech,Success,0.577887
3,funtown,Fail,62.500000
4,funtown,Success,37.500000
5,insight,Fail,4.840325
6,insight,Success,95.159675
7,paytm,Fail,85.721590
8,paytm,Success,14.278410
9,unfamiliar IP,Fail,26.615330


In [213]:
login_success.loc[login_success['LoginSuccess'] == 'Fail', ['login success rate']] = 0

login_success

,Aggregator,LoginSuccess,login success rate
0,aws,Fail,0.000000
1,fintech,Fail,0.000000
2,fintech,Success,0.577887
3,funtown,Fail,0.000000
4,funtown,Success,37.500000
5,insight,Fail,0.000000
6,insight,Success,95.159675
7,paytm,Fail,0.000000
8,paytm,Success,14.278410
9,unfamiliar IP,Fail,0.000000


In [214]:
login_success['login success rate'] =login_success['login success rate'].round(decimals = 2)

login_success_rate = login_success.loc[login_success['LoginSuccess'] == 'Success'].sort_values(by = 'login success rate', ascending = False)

login_success_rate

,Aggregator,LoginSuccess,login success rate
6,insight,Success,95.16
10,unfamiliar IP,Success,73.38
12,youngones,Success,71.01
4,funtown,Success,37.50
8,paytm,Success,14.28
2,fintech,Success,0.58


### Avg transactions per 10 min <a class=anchor id=q5></a>

1. First step was to convert the unix epoch time to pandas datetime format. Since the unix epoch time was 13 digits so the        conversion required milliseconds as the unit.
2. The table was then grouped by the aggregators and the 10 min time interval using pandas resampling function and the            aggregation used was the number of login attempts(transactions) being sent by each aggregator per 10 min interval
3. To get the average I first needed the total number of time intervals over which the total time has been split.
4. Then I calculated the sum of transactions for each aggregator and merged the two tables of total time intervals and the sum    of transactions.
5. The average was calculated by dividing the total transactions by the total time intervals.

In [215]:
df_merged['datetime'] = (pd.to_datetime(df_merged['timestamp'], unit = 'ms'))

df_merged

df_merged_sorted_bytimestamp = df_merged.sort_values(by = 'timestamp')

df_merged_sorted_bytimestamp

df_merged_sorted_bytimestamp.datetime.unique()

array(['2018-10-28T04:13:20.000000000', '2018-10-28T07:00:00.000000000',
       '2018-10-28T09:46:40.000000000'], dtype='datetime64[ns]')

In [110]:
df_resampling = df_merged.groupby('Aggregator').resample('10T', on = 'datetime', label = 'right', closed = 'right').agg({'LoginSuccess': 'count'}).reset_index()
df_resampling.loc[df_resampling['datetime'] == '2018-10-28 09:50:00'] ##checking to see if the time intervals are catching the correct amount of login attempts

,Aggregator,datetime,LoginSuccess
33,aws,2018-10-28 09:50:00,219979
67,fintech,2018-10-28 09:50:00,19977
118,insight,2018-10-28 09:50:00,39459
152,paytm,2018-10-28 09:50:00,8816
186,unfamiliar IP,2018-10-28 09:50:00,65918
220,youngones,2018-10-28 09:50:00,99463


In [216]:
num_rows = df_resampling.groupby('Aggregator').size().reset_index(name = 'total num of time intervals')

num_rows

,Aggregator,total num of time intervals
0,aws,34
1,fintech,34
2,funtown,17
3,insight,34
4,paytm,34
5,unfamiliar IP,34
6,youngones,34


In [217]:
df_resampling.loc[df_resampling['Aggregator'] == 'funtown']

,Aggregator,datetime,LoginSuccess
68,funtown,2018-10-28 04:20:00,1
69,funtown,2018-10-28 04:30:00,0
70,funtown,2018-10-28 04:40:00,0
71,funtown,2018-10-28 04:50:00,0
72,funtown,2018-10-28 05:00:00,0
73,funtown,2018-10-28 05:10:00,0
74,funtown,2018-10-28 05:20:00,0
75,funtown,2018-10-28 05:30:00,0
76,funtown,2018-10-28 05:40:00,0
77,funtown,2018-10-28 05:50:00,0


In [218]:
df_summed_transactions = df_resampling.groupby('Aggregator')['LoginSuccess'].sum().reset_index()

df_summed_transactions

,Aggregator,LoginSuccess
0,aws,516218
1,fintech,54682
2,funtown,8
3,insight,140755
4,paytm,21515
5,unfamiliar IP,96358
6,youngones,219039


In [219]:
avg_transactions = pd.merge(num_rows, df_summed_transactions, how = 'inner', on = ['Aggregator'])

avg_transactions.rename(columns = {'LoginSuccess': 'Total transactions'}, inplace = True)

avg_transactions

,Aggregator,total num of time intervals,Total transactions
0,aws,34,516218
1,fintech,34,54682
2,funtown,17,8
3,insight,34,140755
4,paytm,34,21515
5,unfamiliar IP,34,96358
6,youngones,34,219039


In [220]:
avg_transactions['avg transactions/10 min'] = avg_transactions['Total transactions']/avg_transactions['total num of time intervals']

avg_transactions['avg transactions/10 min'] = avg_transactions['avg transactions/10 min'].round(decimals = 2)

avg_transactions

,Aggregator,total num of time intervals,Total transactions,avg transactions/10 min
0,aws,34,516218,15182.88
1,fintech,34,54682,1608.29
2,funtown,17,8,0.47
3,insight,34,140755,4139.85
4,paytm,34,21515,632.79
5,unfamiliar IP,34,96358,2834.06
6,youngones,34,219039,6442.32


### Max transactions per 10 min <a class=anchor id=q6></a>

The maximum number of transactions were analyzed by grouping the table that was resampled with 10 min time intervals, by the Aggregator and finding the maximum of the transactions each aggregator sent over the 10 min time intervals.

In [221]:
df_max_transactions = df_resampling.groupby('Aggregator')['LoginSuccess'].max().reset_index()

df_max_transactions_sorted = df_max_transactions.sort_values(by = 'LoginSuccess', ascending = False).rename(columns = {'LoginSuccess': 'Max transactions/10 min'})

df_max_transactions_sorted 

,Aggregator,Max transactions/10 min
0,aws,241359
6,youngones,99463
3,insight,80256
5,unfamiliar IP,65918
1,fintech,30454
4,paytm,10199
2,funtown,7


### Impact of CISO proposal <a class=anchor id=q7></a>

1. First I analyzed the data by creating different types of dataframes to see how the time affects the logins.
2. I found there are three unique values of datetime column: 2018-10-28 4:13, 2018-10-28 7:00, 2018-10-28 9:46.
3. While studying the data to find the avg and max number of transactions per 10 min interval, I observed that the above three    times are the only times during the whole time period that the aggregators login to accounts. Funtown was an exception          because it only tried to access the accounts at 2018-10-28 4:13 and 2018-10-28 7:00.
4. Therefore I decide to group the initial merged table by datetime, AccountName, Aggregator, LoginSuccess to get the number of    logins by each aggregator at each of the above three datetimes for each individual account. I did this based on the            information given by the data that at the other 10 min intervals there are no logins by any of the aggregators.
5. I change the value of LoginSuccess from Fail to Success for all values for individual accounts that had a failed attempt        because the information needed is logins not login successes. Hence all logins are assumed to be successful so as to group      the login data by maintaining the account uniqueness as it groups the data by the count of just one type of value instead of    dividing it into two: 'Fail' and 'Success'
6. The sum of logins for individual accounts by each aggregator at the three datetimes was calculated first 
7. The the CISO proposal was used and all the logins greater than 1 for individual accounts was redefined as 1 and the sum was    again calculated. Both these sums were compared to see if the proposal will work. 
8. The comparison showed that the proposal will have a small effect on the account access.  

In [222]:
df_sorted = df_merged.sort_values(by = 'datetime')

df_sorted.loc[df_sorted['datetime'] == '2018-10-28 09:46:40']

,Unnamed: 0_x,IP,LoginSuccess,AccountName,timestamp,Unnamed: 0_y,Aggregator,datetime
897368,897368,54.208.59.10,Fail,7db8ca04aca1566,1.540720e+12,0.0,aws,2018-10-28 09:46:40
897369,897369,54.88.74.128,Fail,6cb697bc9265cb4,1.540720e+12,1.0,aws,2018-10-28 09:46:40
897395,897395,54.208.59.10,Fail,68140df6bf26123,1.540720e+12,0.0,aws,2018-10-28 09:46:40
897371,897371,68.142.133.182,Success,06edca2a0f27b56,1.540720e+12,NaN,unfamiliar IP,2018-10-28 09:46:40
897372,897372,34.230.198.27,Fail,4fd5434424f3802,1.540720e+12,16.0,fintech,2018-10-28 09:46:40
...,...,...,...,...,...,...,...,...
746161,746161,216.34.61.237,Success,cff86baf0e7c7fd,1.540720e+12,114.0,youngones,2018-10-28 09:46:40
746160,746160,64.41.181.97,Fail,561a33ba2b4251d,1.540720e+12,59.0,youngones,2018-10-28 09:46:40
746159,746159,216.34.61.119,Success,8369641815e3192,1.540720e+12,202.0,youngones,2018-10-28 09:46:40
746170,746170,64.41.181.163,Success,a1e40a33ca721ee,1.540720e+12,64.0,youngones,2018-10-28 09:46:40


In [223]:
user_account_accessed =  pd.merge(num_rows, total_user_acc_accessed, how = 'inner', on = ['Aggregator'])

user_account_accessed

,Aggregator,total num of time intervals,total accounts accessed
0,aws,34,0
1,fintech,34,316
2,funtown,17,3
3,insight,34,133942
4,paytm,34,3072
5,unfamiliar IP,34,70712
6,youngones,34,155549


In [224]:
df_accounts = df_merged.groupby(['AccountName', 'Aggregator','LoginSuccess'])['LoginSuccess'].count().reset_index(name = 'logins')

df_accounts.head(10)

df_accounts.loc[df_accounts['LoginSuccess'] == 'Fail', ['logins']] = 0

df_accounts.head(10)


,AccountName,Aggregator,LoginSuccess,logins
0,00001e0d32671b7,insight,Success,1
1,0000358930daab8,insight,Success,1
2,00003feb8e56213,fintech,Fail,0
3,00004e6dc972815,aws,Fail,0
4,00006c4428f52f1,aws,Fail,0
5,00009734198ee5a,aws,Fail,0
6,00009b8a2d97d2c,insight,Success,1
7,00009cbbed5ec25,insight,Success,1
8,0000b24d75f95ba,aws,Fail,0
9,0000d2acf418606,youngones,Success,1


In [225]:
df_accounts_accessed = df_accounts.groupby(['Aggregator', 'AccountName'])['logins'].sum().reset_index()

df_accounts_accessed

,Aggregator,AccountName,logins
0,aws,00004e6dc972815,0
1,aws,00006c4428f52f1,0
2,aws,00009734198ee5a,0
3,aws,0000b24d75f95ba,0
4,aws,000115cd8b57a41,0
...,...,...,...
933255,youngones,fffdfe48698965c,1
933256,youngones,fffea5b4641e04c,1
933257,youngones,fffea75f28fff68,1
933258,youngones,ffffd8908ddd2b7,1


In [226]:
 df_accounts_accessed.loc[df_accounts_accessed.logins == df_accounts_accessed.logins.max()]

df_accounts_accessed.loc[df_accounts_accessed.logins > 10].Aggregator.unique()

a = df_accounts_accessed.loc[(df_accounts_accessed.Aggregator == 'unfamiliar IP') & (df_accounts_accessed.logins > 1)]

a

#a.logins.sum()/2240 finding avg logins per account

#2.286607142857143/34 : finding avg. logins per account per 10 min interval by using 34 intervals

,Aggregator,AccountName,logins
698250,unfamiliar IP,000169809683cc8,2
698284,unfamiliar IP,001bf721dc174d5,2
698285,unfamiliar IP,001ce055803c4e5,2
698501,unfamiliar IP,00ae4f26a68c892,8
698518,unfamiliar IP,00bc40450175ef7,2
...,...,...,...
788381,unfamiliar IP,ff7eeadec2d7be1,2
788418,unfamiliar IP,ffa35488fa5a147,2
788435,unfamiliar IP,ffaed88eb1c49f9,2
788488,unfamiliar IP,ffd3b991fb2498a,2


### unfamiliar IP are accessing over 2240 accounts more than 1 time during the total time interval which averages around to 2.2866 logins per account.
This comes around to 0.0673 avg logins per account per 10 min interval

In [227]:
df_timegrouped = df_merged.groupby(['Aggregator', 'datetime', 'AccountName', 'LoginSuccess'])['LoginSuccess'].count().reset_index(name = 'total transactions')

df_timegrouped

,Aggregator,datetime,AccountName,LoginSuccess,total transactions
0,aws,2018-10-28 04:13:20,00009734198ee5a,Fail,1
1,aws,2018-10-28 04:13:20,000115cd8b57a41,Fail,1
2,aws,2018-10-28 04:13:20,0002748228e839f,Fail,1
3,aws,2018-10-28 04:13:20,00030846d70f7ba,Fail,1
4,aws,2018-10-28 04:13:20,0003722e9dd113c,Fail,1
...,...,...,...,...,...
1008478,youngones,2018-10-28 09:46:40,fffdc19587f85c9,Success,1
1008479,youngones,2018-10-28 09:46:40,fffdeb901d01a78,Success,1
1008480,youngones,2018-10-28 09:46:40,fffea5b4641e04c,Fail,1
1008481,youngones,2018-10-28 09:46:40,fffea5b4641e04c,Success,1


In [141]:
df_timegrouped.loc[(df_timegrouped['total transactions'] > 1) & (df_timegrouped['Aggregator'] == 'unfamiliar IP')]

,Aggregator,datetime,AccountName,LoginSuccess,total transactions
709114,unfamiliar IP,2018-10-28 04:13:20,0059df55c9da632,Fail,2
709119,unfamiliar IP,2018-10-28 04:13:20,00ce1d92a2cb7c4,Fail,2
709122,unfamiliar IP,2018-10-28 04:13:20,00fa96a47abc097,Fail,2
709141,unfamiliar IP,2018-10-28 04:13:20,02880feb662132b,Fail,2
709144,unfamiliar IP,2018-10-28 04:13:20,02d55794fb0026a,Fail,2
...,...,...,...,...,...
800307,unfamiliar IP,2018-10-28 09:46:40,ff6ccfed26dc6e8,Fail,2
800330,unfamiliar IP,2018-10-28 09:46:40,ff8daadfb1e4a62,Fail,2
800346,unfamiliar IP,2018-10-28 09:46:40,ff9ec52be7783aa,Fail,2
800351,unfamiliar IP,2018-10-28 09:46:40,ffa35488fa5a147,Success,2


In [166]:
df_accounts_hits = df_merged.groupby(['datetime', 'AccountName', 'Aggregator','LoginSuccess'])['LoginSuccess'].count().reset_index(name = 'transactions')

df_accounts_hits

,datetime,AccountName,Aggregator,LoginSuccess,transactions
0,2018-10-28 04:13:20,00001e0d32671b7,insight,Success,1
1,2018-10-28 04:13:20,00009734198ee5a,aws,Fail,1
2,2018-10-28 04:13:20,00009b8a2d97d2c,insight,Success,1
3,2018-10-28 04:13:20,00009cbbed5ec25,insight,Success,1
4,2018-10-28 04:13:20,000115cd8b57a41,aws,Fail,1
...,...,...,...,...,...
1008478,2018-10-28 09:46:40,ffff397cb259b22,aws,Fail,1
1008479,2018-10-28 09:46:40,ffff42d29c081b8,aws,Fail,1
1008480,2018-10-28 09:46:40,ffff47de5009cd8,unfamiliar IP,Success,1
1008481,2018-10-28 09:46:40,ffff87d0bd98a4e,aws,Fail,1


In [151]:
df_accounts_hits.loc[df_accounts_hits['LoginSuccess'] == 'Fail', 'LoginSuccess'] = 'Success'

In [171]:
df_accounts_logins = df_accounts_hits.groupby(['datetime', 'AccountName', 'Aggregator'])['LoginSuccess'].count().reset_index()

df_accounts_logins.tail(30)


,datetime,AccountName,Aggregator,LoginSuccess
949760,2018-10-28 09:46:40,fffd369b1c647a4,youngones,1
949761,2018-10-28 09:46:40,fffd493d83592cf,aws,1
949762,2018-10-28 09:46:40,fffd7340b129ba7,fintech,1
949763,2018-10-28 09:46:40,fffd7e54ed329e6,unfamiliar IP,1
949764,2018-10-28 09:46:40,fffd84fa91da455,youngones,2
949765,2018-10-28 09:46:40,fffdc19587f85c9,youngones,2
949766,2018-10-28 09:46:40,fffdeb901d01a78,youngones,1
949767,2018-10-28 09:46:40,fffdfc52feabd59,unfamiliar IP,1
949768,2018-10-28 09:46:40,fffe20ed67ef34c,aws,1
949769,2018-10-28 09:46:40,fffe20ed67ef34c,fintech,1


In [173]:
datetime_aggregator = df_accounts_logins.groupby(['datetime', 'Aggregator'])['LoginSuccess'].sum().reset_index()

datetime_aggregator

,datetime,Aggregator,LoginSuccess
0,2018-10-28 04:13:20,aws,53065
1,2018-10-28 04:13:20,fintech,4163
2,2018-10-28 04:13:20,funtown,1
3,2018-10-28 04:13:20,insight,20080
4,2018-10-28 04:13:20,paytm,2415
5,2018-10-28 04:13:20,unfamiliar IP,2808
6,2018-10-28 04:13:20,youngones,20890
7,2018-10-28 07:00:00,aws,233865
8,2018-10-28 07:00:00,fintech,29984
9,2018-10-28 07:00:00,funtown,5


In [175]:
hits_one_per_aggregator  = df_accounts_logins.copy()

hits_one_per_aggregator.loc[hits_one_per_aggregator['LoginSuccess'] > 1, 'LoginSuccess'] = 1

hits_one_per_aggregator

,datetime,AccountName,Aggregator,LoginSuccess
0,2018-10-28 04:13:20,00001e0d32671b7,insight,1
1,2018-10-28 04:13:20,00009734198ee5a,aws,1
2,2018-10-28 04:13:20,00009b8a2d97d2c,insight,1
3,2018-10-28 04:13:20,00009cbbed5ec25,insight,1
4,2018-10-28 04:13:20,000115cd8b57a41,aws,1
...,...,...,...,...
949785,2018-10-28 09:46:40,ffff397cb259b22,aws,1
949786,2018-10-28 09:46:40,ffff42d29c081b8,aws,1
949787,2018-10-28 09:46:40,ffff47de5009cd8,unfamiliar IP,1
949788,2018-10-28 09:46:40,ffff87d0bd98a4e,aws,1


In [177]:
hits_one_per_aggregators =hits_one_per_aggregator.groupby(['datetime', 'Aggregator'])['LoginSuccess'].sum().reset_index()

hits_one_per_aggregators

,datetime,Aggregator,LoginSuccess
0,2018-10-28 04:13:20,aws,53065
1,2018-10-28 04:13:20,fintech,4162
2,2018-10-28 04:13:20,funtown,1
3,2018-10-28 04:13:20,insight,20047
4,2018-10-28 04:13:20,paytm,2412
5,2018-10-28 04:13:20,unfamiliar IP,2803
6,2018-10-28 04:13:20,youngones,15000
7,2018-10-28 07:00:00,aws,233865
8,2018-10-28 07:00:00,fintech,29955
9,2018-10-28 07:00:00,funtown,4
